In [1]:
import cv2
import os
import csv
import glob

In [2]:
# Folders and Files
image_folder = r"C:\Users\agah2\Desktop\JupyterNotebooks\BIP\BIP_images_roboter_cam_allinone\all"   # Folder where your images are stored
csv_file = 'labels.csv'    # Output file

In [3]:
def click_event(event, x, y, flags, params):
    """
    Mouse callback function: 
    - Left Click: Records coordinates, draws a circle, and saves to CSV.
    """
    image = params['image']
    img_name = params['img_name']
    height, width, _ = image.shape

    if event == cv2.EVENT_LBUTTONDOWN:
        # 1. Normalize coordinates (0.0 to 1.0)
        # This makes the model independent of specific image resolutions
        x_norm = x / width
        y_norm = y / height

        # 2. Visual Feedback: Draw a green circle where you clicked
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow('Labeling Tool', image)

        # 3. Save to CSV immediately
        # Mode 'a' appends to the file so you don't lose progress if it crashes
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            # Format: filename, x_norm, y_norm
            writer.writerow([img_name, x_norm, y_norm])
        
        print(f"Recorded: {img_name} -> X: {x_norm:.3f}, Y: {y_norm:.3f}")

In [4]:
def main():
    # Check if image folder exists
    if not os.path.exists(image_folder):
        print(f"Error: Folder '{image_folder}' not found. Please create it and put your images inside.")
        return

    # Create CSV file with header if it doesn't exist
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['filename', 'x', 'y'])

    # Load all jpg/png images
    # We sort them to ensure consistent order
    image_paths = sorted(glob.glob(os.path.join(image_folder, '*.jpg')) + 
                         glob.glob(os.path.join(image_folder, '*.png')))

    # Read existing labels to skip already labeled images (Resume capability)
    existing_labels = set()
    if os.path.exists(csv_file):
        with open(csv_file, 'r') as f:
            reader = csv.reader(f)
            next(reader, None) # Skip header
            for row in reader:
                if row:
                    existing_labels.add(row[0])

    print(f"Found {len(image_paths)} images.")
    print(f"Already labeled: {len(existing_labels)}")
    print("-------------------------------------------------")
    print("CONTROLS:")
    print("  [Left Click] : Mark the target path center.")
    print("  [Any Key]    : Skip image (if bad/blurry).")
    print("  [ESC]        : Quit.")
    print("-------------------------------------------------")

    for img_path in image_paths:
        img_name = os.path.basename(img_path)

        # Skip if already done
        if img_name in existing_labels:
            continue

        image = cv2.imread(img_path)
        if image is None:
            print(f"Could not read {img_path}")
            continue

        cv2.imshow('Labeling Tool', image)
        
        # Pass image and name to the callback so we can save them
        params = {'image': image, 'img_name': img_name}
        cv2.setMouseCallback('Labeling Tool', click_event, params)

        # Wait for a key press
        # If you click, the callback handles saving.
        # Pressing a key moves to the next image.
        key = cv2.waitKey(0) 

        if key == 27: # ESC key to exit
            print("Exiting...")
            break

    cv2.destroyAllWindows()
    print("Done! Labels saved to", csv_file)

if __name__ == "__main__":
    main()

Found 435 images.
Already labeled: 0
-------------------------------------------------
CONTROLS:
  [Left Click] : Mark the target path center.
  [Any Key]    : Skip image (if bad/blurry).
  [ESC]        : Quit.
-------------------------------------------------


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
